<a href="https://colab.research.google.com/github/Loga19818eeanvesh/CNN_Models/blob/main/2014_Inception_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow==2.2.0

     |████████████████████████████████| 516.2 MB 4.2 kB/s 
     |████████████████████████████████| 454 kB 50.9 MB/s 
     |████████████████████████████████| 2.9 MB 47.1 MB/s 
     |████████████████████████████████| 3.0 MB 50.8 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.5.3
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.8.0
    Uninstalli

In [3]:
import tensorflow as tf 
print(tf.__version__)

2.2.0


In [11]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D,MaxPool2D,GlobalAveragePooling2D,Dense,Concatenate,Flatten,Dropout,AveragePooling2D
from tensorflow.keras.activations import relu,softmax

In [7]:
class InceptionModule(Model):
  def __init__(self,filters_1by1,filters_1by1_3,filters_1by1_5,filters_1by1_p,filters_3by3,filters_5by5):
    super().__init__()
    self.conv2d_1by1 = Conv2D(filters=filters_1by1,kernel_size=1,activation=relu)
    self.conv2d_1by1_3 = Conv2D(filters=filters_1by1_3,kernel_size=1,activation=relu)
    self.conv2d_1by1_5 = Conv2D(filters=filters_1by1_5,kernel_size=1,activation=relu)
    self.conv2d_1by1_p = Conv2D(filters=filters_1by1_p,kernel_size=1,activation=relu)
    self.conv2d_3by3 = Conv2D(filters=filters_3by3,kernel_size=3,activation=relu,padding='same')
    self.conv2d_5by5 = Conv2D(filters=filters_5by5,kernel_size=5,activation=relu,padding='same')
    self.max_pool = MaxPool2D(pool_size=(3, 3), strides=1, padding='same')
    self.concat = Concatenate(axis = -1)

  def call(self,inputs):
    x_1by1 = self.conv2d_1by1(inputs)

    x_3by3 = self.conv2d_1by1_3(inputs)
    x_3by3 = self.conv2d_3by3(x_3by3)

    x_5by5 = self.conv2d_1by1_5(inputs)
    x_5by5 = self.conv2d_5by5(x_5by5)

    x_p = self.max_pool(inputs)
    x_p = self.conv2d_1by1_p(x_p)

    x = self.concat([x_1by1,x_3by3,x_5by5,x_p])

    return x


In [27]:
class InceptionNetwork(Model):
  def __init__(self,input_shape,noof_classes):
    super().__init__()
    self.conv1 = Conv2D(input_shape=input_shape,filters=64, kernel_size=7, strides=2, padding='valid')
    self.conv2 = Conv2D(filters=192, kernel_size=3, padding='same')
    self.maxpool = MaxPool2D(pool_size=(3, 3), strides=2, padding='valid')
    self.inception_3a = InceptionModule(filters_1by1=64,filters_1by1_3=96,filters_1by1_5=16,filters_1by1_p=32,filters_3by3=128,filters_5by5=32)
    self.inception_3b = InceptionModule(filters_1by1=128,filters_1by1_3=128,filters_1by1_5=32,filters_1by1_p=64,filters_3by3=192,filters_5by5=96)
    self.inception_4a = InceptionModule(filters_1by1=192,filters_1by1_3=96,filters_3by3=208,filters_1by1_5=16,filters_5by5=48,filters_1by1_p=64)
    self.inception_4b = InceptionModule(filters_1by1=160,filters_1by1_3=112,filters_3by3=224,filters_1by1_5=24,filters_5by5=64,filters_1by1_p=64)
    self.inception_4c = InceptionModule(filters_1by1=128,filters_1by1_3=128,filters_3by3=256,filters_1by1_5=24,filters_5by5=64,filters_1by1_p=64)
    self.inception_4d = InceptionModule(filters_1by1=112,filters_1by1_3=144,filters_3by3=288,filters_1by1_5=32,filters_5by5=64,filters_1by1_p=64)
    self.inception_4e = InceptionModule(filters_1by1=256,filters_1by1_3=160,filters_3by3=320,filters_1by1_5=32,filters_5by5=128,filters_1by1_p=128)
    self.inception_5a = InceptionModule(filters_1by1=256,filters_1by1_3=160,filters_3by3=320,filters_1by1_5=32,filters_5by5=128,filters_1by1_p=128)
    self.inception_5b = InceptionModule(filters_1by1=384,filters_1by1_3=192,filters_3by3=384,filters_1by1_5=48,filters_5by5=128,filters_1by1_p=128)
    self.globalavgpool = GlobalAveragePooling2D()
    self.flatten = Flatten()
    self.dropout = Dropout(rate=0.4)
    self.classifier = Dense(units=noof_classes,activation=softmax)
    self.avgpool_o1 =  AveragePooling2D(pool_size=(5, 5), strides=3, padding='valid')
    self.conv_1by1_o1 = Conv2D(filters=128,kernel_size=1,activation=relu)
    self.dense1_o1 = Dense(units=noof_classes,activation=relu)
    self.dense2_o1 = Dense(units=noof_classes,activation=softmax)
    self.dropout_o = Dropout(rate=0.7)
    self.avgpool_o2 =  AveragePooling2D(pool_size=(5, 5), strides=3, padding='valid')
    self.conv_1by1_o2 = Conv2D(filters=128,kernel_size=1,activation=relu)
    self.dense1_o2 = Dense(units=noof_classes,activation=relu)
    self.dense2_o2 = Dense(units=noof_classes,activation=softmax)
    
  def call(self,inputs):
    x = self.conv1(inputs)
    x = self.maxpool(x)
    x = self.conv2(x)
    x = self.maxpool(x)
    x = self.inception_3a(x)
    x = self.inception_3b(x)
    x = self.maxpool(x)

    x_4a = self.inception_4a(x)

    x = self.inception_4b(x_4a)
    x = self.inception_4c(x)

    x_4d = self.inception_4d(x)

    x = self.inception_4e(x_4d)
    x = self.maxpool(x)
    x = self.inception_5a(x)
    x = self.inception_5b(x)
    x = self.globalavgpool(x)
    x = self.flatten(x)
    x = self.dropout(x)

    output = self.classifier(x)

    o1 = self.avgpool_o1(x_4a)
    o1 = self.conv_1by1_o1(o1)
    o1 = self.flatten(o1)
    o1 = self.dense1_o1(o1)
    o1 = self.dropout_o(o1)
    output1 = self.dense2_o1(o1)

    o2 = self.avgpool_o2(x_4d)
    o2 = self.conv_1by1_o2(o2)
    o2 = self.flatten(o2)
    o2 = self.dense1_o2(o2)
    o2 = self.dropout_o(o2)
    output2 = self.dense2_o2(o2)

    return [output,output1,output1]


In [28]:
InceptionModel = InceptionNetwork(input_shape=(None,224,224,3),noof_classes=1000)


In [29]:
InceptionModel.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                       loss=[tf.keras.losses.SparseCategoricalCrossentropy(),tf.keras.losses.SparseCategoricalCrossentropy(),tf.keras.losses.SparseCategoricalCrossentropy()],
                       metrics=[tf.keras.metrics.SparseCategoricalAccuracy(),tf.keras.metrics.SparseCategoricalAccuracy(),tf.keras.metrics.SparseCategoricalAccuracy()],
                       loss_weights=[1,0.3,0.3])


In [30]:
InceptionModel.build(input_shape=(None,224,224,3)) 
InceptionModel.summary()

Model: "inception_network_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_174 (Conv2D)          multiple                  9472      
_________________________________________________________________
conv2d_175 (Conv2D)          multiple                  110784    
_________________________________________________________________
max_pooling2d_30 (MaxPooling multiple                  0         
_________________________________________________________________
inception_module_27 (Incepti multiple                  163696    
_________________________________________________________________
inception_module_28 (Incepti multiple                  388736    
_________________________________________________________________
inception_module_29 (Incepti multiple                  376176    
_________________________________________________________________
inception_module_30 (Incepti multiple          